# Deep Learning

### DL imports

In [44]:
# just to make sure that all libraries are available
%pip install tensorflow matplotlib pandas folium kaggle scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 31.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.7/28.7 MB 59.2 MB/s eta 0:00:0000:0100:01
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [45]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

### Misc. Imports

In [23]:
import matplotlib.pyplot as plt
import pandas as pd
import json
import os
import math
import string

## Get data from Kaggle

###! Remember add the kaggle.json file to the visible files

In [4]:
os.environ['KAGGLE_USERNAME'] = "x"
os.environ['KAGGLE_KEY'] = "x"

In [6]:
!kaggle datasets download -d yelp-dataset/yelp-dataset

100%|█████████████████████████████████████▉| 4.07G/4.07G [03:03<00:00, 27.8MB/s]
100%|██████████████████████████████████████| 4.07G/4.07G [03:03<00:00, 23.8MB/s]


In [7]:
!mkdir yelp-data
!unzip -d yelp-data yelp-dataset.zip

Archive:  yelp-dataset.zip
  inflating: yelp-data/Dataset_User_Agreement.pdf  
  inflating: yelp-data/yelp_academic_dataset_business.json  
  inflating: yelp-data/yelp_academic_dataset_checkin.json  
  inflating: yelp-data/yelp_academic_dataset_review.json  
  inflating: yelp-data/yelp_academic_dataset_tip.json  
  inflating: yelp-data/yelp_academic_dataset_user.json  


## Examine and plot it

In [8]:
data_dir = "yelp-data/" # splitting data up like this might be useful for later (in case i want to use other files)
target_file = "yelp_academic_dataset_review.json"

In [9]:
data_file = data_dir + target_file

In [10]:
def read_to_dataframe(data_file, max_nl = 100_000):
  data = []
  with open(data_file) as f:
    nl = 0
  
    for line in f:
      data.append(json.loads(line))
    
      nl += 1
      if nl >= max_nl:
        break

  return pd.DataFrame(data)

In [11]:
data_df = read_to_dataframe(data_file)

In [12]:
data_df.head() # to get an idea as to what we're dealing with

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5.0,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3.0,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15


In [13]:
data_df.info() # all entries are non-null! 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   review_id    100000 non-null  object 
 1   user_id      100000 non-null  object 
 2   business_id  100000 non-null  object 
 3   stars        100000 non-null  float64
 4   useful       100000 non-null  int64  
 5   funny        100000 non-null  int64  
 6   cool         100000 non-null  int64  
 7   text         100000 non-null  object 
 8   date         100000 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 6.9+ MB


In [14]:
data_df.describe()

,stars,useful,funny,cool
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,3.842910,0.898980,0.257180,0.347030
std,1.354729,2.205186,1.010212,1.066382
min,1.000000,0.000000,0.000000,0.000000
25%,3.000000,0.000000,0.000000,0.000000
50%,4.000000,0.000000,0.000000,0.000000
75%,5.000000,1.000000,0.000000,0.000000
max,5.000000,320.000000,98.000000,49.000000


I don't particularly like the identifiers used in this dataframe, so I'll join this data with the data containing additional information about them so that we get much more meaningful and human-friendly data.

In [38]:
business_file = "yelp_academic_dataset_business.json"

business_df = read_to_dataframe(data_dir + business_file, max_nl=math.inf) # no need to set a limit (the file is 113MB large)
business_df.drop(["stars"], axis=1, inplace=True) # this is the rating of the businesses (we don't care about that)

In [39]:
rich_data = pd.merge(data_df, business_df, on="business_id")

In [40]:
rich_data.head() # as we can see, we now have a richer dataset, which makes it easier to identify the businesses (as far as the reviews and the users are concerned, it's not important to further enrich the data)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,name,...,city,state,postal_code,latitude,longitude,review_count,is_open,attributes,categories,hours
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11,Turning Point of North Wales,...,North Wales,PA,19454,40.210196,-75.223639,169,1,"{'NoiseLevel': 'u'average'', 'HasTV': 'False',...","Restaurants, Breakfast & Brunch, Food, Juice B...","{'Monday': '7:30-15:0', 'Tuesday': '7:30-15:0'..."
1,VJxlBnJmCDIy8DFG0kjSow,Iaee7y6zdSB3B-kRCo4z1w,XQfwVwDr-v0ZS3_CbbE5Xw,2.0,0,0,0,This is the second time we tried turning point...,2017-05-13 17:06:55,Turning Point of North Wales,...,North Wales,PA,19454,40.210196,-75.223639,169,1,"{'NoiseLevel': 'u'average'', 'HasTV': 'False',...","Restaurants, Breakfast & Brunch, Food, Juice B...","{'Monday': '7:30-15:0', 'Tuesday': '7:30-15:0'..."
2,S6pQZQocMB1WHMjTRbt77A,ejFxLGqQcWNLdNByJlIhnQ,XQfwVwDr-v0ZS3_CbbE5Xw,4.0,2,0,1,The place is cute and the staff was very frien...,2017-08-08 00:58:18,Turning Point of North Wales,...,North Wales,PA,19454,40.210196,-75.223639,169,1,"{'NoiseLevel': 'u'average'', 'HasTV': 'False',...","Restaurants, Breakfast & Brunch, Food, Juice B...","{'Monday': '7:30-15:0', 'Tuesday': '7:30-15:0'..."
3,WqgTKVqWVHDHjnjEsBvUgg,f7xa0p_1V9lx53iIGN5Sug,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,We came on a Saturday morning after waiting a ...,2017-11-19 02:20:23,Turning Point of North Wales,...,North Wales,PA,19454,40.210196,-75.223639,169,1,"{'NoiseLevel': 'u'average'', 'HasTV': 'False',...","Restaurants, Breakfast & Brunch, Food, Juice B...","{'Monday': '7:30-15:0', 'Tuesday': '7:30-15:0'..."
4,M0wzFFb7pefOPcxeRVbLag,dCooFVCk8M1nVaQqcfTL3Q,XQfwVwDr-v0ZS3_CbbE5Xw,2.0,0,0,0,"Mediocre at best. The decor is very nice, and ...",2017-09-09 17:49:47,Turning Point of North Wales,...,North Wales,PA,19454,40.210196,-75.223639,169,1,"{'NoiseLevel': 'u'average'', 'HasTV': 'False',...","Restaurants, Breakfast & Brunch, Food, Juice B...","{'Monday': '7:30-15:0', 'Tuesday': '7:30-15:0'..."


For a much cooler visualization, we can use the folium library!

In [ ]:
import folium

fol_fig = folium.Figure(width=1000, height=600)

# Create a map object centered at a specific location (e.g., the geographical center of North America (we know the data is from USA and Canada))
map_obj = folium.Map(location=[39.7392, -104.9903], zoom_start=4, tiles='Stamen Toner').add_to(fol_fig)

sampled_df = rich_data.sample(n=1000) # to speed up execution (up to 1000 works fine, above that it starts to slow down significantly)
# Iterate over your dataset and add markers for each data point
for index, row in sampled_df.iterrows():
    # Extract the latitude and longtude values from your dataframe
    latitude = row['latitude']
    longitude = row['longitude']
    
    # Add a marker to the map for each data point
    folium.Marker([latitude, longitude]).add_to(map_obj)

# Display the map
map_obj

## Prepare the data for trainingd validation sets, make labels, tokenize, etc 

In [47]:
# Load your dataset into a pandas DataFrame (assuming 'df' is your DataFrame)
# Split the dataset into features (X) and labels (y)
X = rich_data['text']
y = rich_data['stars']

# Split the dataset into training (70%), validation (15%), and testing (15%) sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, stratify=y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1765, stratify=y_train_val, random_state=42)

print("Training set shape:", X_train.shape, y_train.shape)
print("Validation set shape:", X_val.shape, y_val.shape)
print("Testing set shape:", X_test.shape, y_test.shape)

Training set shape: (69997,) (69997,)
Validation set shape: (15003,) (15003,)
Testing set shape: (15000,) (15000,)


In [55]:
# we have the very convenient TextVectorizatoin layer for preprocessing!
text_vectorization = layers.TextVectorization(
    max_tokens=20_000,
    output_mode='int', # encode words as integers
    standardize="lower_and_strip_punctuation",
    split="whitespace"
)

In [62]:
# Create TensorFlow datasets for the training, validatoin and test data
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [57]:
text_vectorization.adapt(train_dataset.map(lambda review, label: review)) # build the vocabulary (might take a while...)

In [63]:
# Define the preprocessing pipeline for the datasets
def preprocess(review, label):
    vectorized_review = text_vectorization(review)
    return vectorized_review, label

# Apply the preprocessing pipeline to the datasets
train_dataset = train_dataset.map(preprocess, num_parallel_calls=4) # adjust the parallel calls as needed
val_dataset = val_dataset.map(preprocess, num_parallel_calls=4)
test_dataset = test_dataset.map(preprocess, num_parallel_calls=4)

# Shuffle and batch the training dataset
batch_size = 32  # Adjust this value based on your requirements
train_dataset = train_dataset.shuffle(len(X_train)).batch(batch_size)

# Batch the validation and testing datasets
val_dataset = val_dataset.batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

## Find correct architecture

## Train

## Evaluate